In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

Can i write something which checks if it is stable? Say check the difference between the final two values say?  
have a feeling there will be an error some where. maybe the way i calculate the resistance shouldnt be for all. think about the C line.  
have worked out that it has to be under 3 delay with any alpha value.

#### data input

In [10]:
baselines = pd.read_csv(r"C:\Users\timst\Onedrive - Nexus365\4YP\python\_storage\baselines.csv", index_col=0).rename(columns={"Q in single(m^3/s????)": "baseline Q"})


#### paramaters

In [11]:
optimised_value_tau = 2.292929292929293
phi_min_baseline = 0.153
n=1
max_time = 200
no = 101
delay_pressure_drop = 15

#### work out vectors

In [12]:
t = pd.Series(np.linspace(0,max_time,no))
dt = max_time/(no-1)

#### PRessure stuff
pressure_in = pd.Series(np.zeros(len(t)))
ratio_drop = 0.5
for i in range(len(t)):
    if t[i] <= delay_pressure_drop:
        pressure_in[i] = 60
    else:
        pressure_in[i] = 60 - 34.18*ratio_drop
pressure_out = pd.Series(np.zeros(len(t))) + 60 - 34.18
pressure_difference = pressure_in - pressure_out

## Model

#### Functions

In [13]:
def total_R(baselines,phi,alpha):
    C_ = alpha*(baselines.loc[6,'Resistance for Q']/2)/phi**4
    C_6 = (C_ + baselines.loc[5,'Resistance for Q'] + baselines.loc[7,'Resistance for Q'])/2
    C_65 = (C_6 + baselines.loc[4,'Resistance for Q'] + baselines.loc[8,'Resistance for Q'])/2
    C_654 = (C_65 + baselines.loc[3,'Resistance for Q'] + baselines.loc[9,'Resistance for Q'])/2
    C_6543 = (C_654 + baselines.loc[2,'Resistance for Q'] + baselines.loc[10,'Resistance for Q'])/2
    C_65432 = (C_6543 + baselines.loc[1,'Resistance for Q'] + baselines.loc[11,'Resistance for Q'])/2
    C_654321 = C_65432 + baselines.loc[0,'Resistance for Q'] + baselines.loc[12,'Resistance for Q']
    R_total = C_654321
    return R_total

def check_imag_roots_real(all_roots):
    counter = 0
    sols = 0
    for i in range(len(all_roots)):
        if all_roots[i].imag <= 10e-13 and all_roots[i].imag >= -10e-13:
            sols = all_roots[i].real
            counter += 1
    if counter != 1:
        print('Something seems wrong with the partial presure to Saturation cubic solver as not one solution is returned.')
    return sols
 
for i in range(len(df['Saturation'])):
    root_temp = None
    sols = 0
    root_temp = np.roots([1,0,150,23400*df.loc[i,'Saturation']/(df.loc[i,'Saturation']-1)]) #gives mmHg , *133.322
    sols = check_imag_roots_real(root_temp) 
    #sols = sols*133.322 ###Remove this step if want in mmHg
    df.loc[i,'partial pressure blood(mmHg)'] = sols

def euler(alpha,delay):
    delay_number = round(delay/dt)
    
    R_tot = pd.Series(np.zeros(len(t)))
    Q_tot = pd.Series(np.zeros(len(t)))
    Q_norm = pd.Series(np.ones(len(t)))
    
    phi = pd.Series(np.ones(no))
    phi_min = pd.Series(np.ones(no))
    dphidt = pd.Series(np.zeros(no))
    
    for i in range(len(t)):
        
        if i >=delay_number: 
            R_tot[i] = total_R(baselines,phi[i],alpha)
            Q_tot[i] = pressure_difference[i]/R_tot[i]
            Q_norm[i] = Q_tot[i] / baselines['baseline Q'][0]
            dphidt[i] =  (1/optimised_value_tau)*( -phi[i-delay_number] + Q_norm[i]*(1-phi_min[i]) + phi_min[i] )
            phi[i+1] = dphidt[i]*dt + phi[i]
            phi_min[i+1] = 1-(1-phi_min_baseline)*(1-Q_norm[i])**n
            
    combined = pd.DataFrame()
    combined['t'] = t
    combined['pressure_difference'] = pressure_difference
    combined['phi'] = phi
    combined['phi_min'] = phi_min
    combined['dphidt'] = dphidt
    combined['R_tot'] = R_tot
    combined['Q_tot'] = Q_tot
    combined['Q_norm'] = Q_norm
            
    return combined

NameError: name 'df' is not defined

#### the loop

In [20]:
response = {}
# alpha = np.linspace(0,1,num=10)
# delay = np.linspace(2.99,3, num=10)
alpha = 0.8
delay = 2.999999

# for i in range(len(alpha)):
#     for j in range(len(delay)):
        #first is alpha second is delay
#             response[i,j] = euler(alpha[i],delay[j])

out = euler(alpha,delay)

#### Vessel details
In order to find the partials in tissue, we need to find the vessel details at each time step. As before we were just considering it as a whole to find some parameters for our capillaries.  
another idea i have had is that you could just take the end values, but this doesnt seem as full proof a method tbh. will be able to get some good plots from this.
/n
whats going on with viscocity?
/n
need to update the resistances, pressure drop, resiatnces for Q, IN ORDER TO CHANGE partial pressure blood therefore partial pressure tissues  
saturations maybe, 

basically need to speak to payne to find out what stuff stays constant and what changes.

In [35]:
vessel_data_alltime = {}
vessel_values_general = baselines.drop(['Viscocity(mPAs)','baseline Q','Resistance for U', 'U in single(m/s????)'],axis=1)

In [69]:
for i in range(len(t)):
    vessel_values_for_each_step = vessel_values_general
    ####### hang on shit, is this the correct way to calculate the resistance? isnt it a certain amount the same and some change?
    vessel_values_for_each_step.loc[6,'Resistance for Q '] = (1-alpha)*(baselines.loc[6,'Resistance for Q']/2) + alpha*(baselines.loc[6,'Resistance for Q']/2)/out.loc[i,'phi']**4
    vessel_data_alltime[i] = vessel_values_for_each_step

In [33]:
out

,t,pressure_difference,phi,phi_min,dphidt,R_tot,Q_tot,Q_norm
0,0.0,34.18,1.000000,1.000000,0.000000e+00,0.000000,0.000000,1.000000
1,2.0,34.18,1.000000,1.000000,0.000000e+00,1.208563,28.281512,1.015165
2,4.0,34.18,1.000000,1.012845,-8.495471e-05,1.208563,28.281512,1.015165
3,6.0,34.18,0.999830,1.012845,-8.471409e-05,1.208615,28.280316,1.015122
4,8.0,34.18,0.999661,1.012809,-1.013326e-05,1.208666,28.279122,1.015079
...,...,...,...,...,...,...,...,...
96,192.0,17.09,0.735553,0.526728,9.632171e-09,1.390285,12.292444,0.441237
97,194.0,17.09,0.735553,0.526728,6.108649e-09,1.390285,12.292444,0.441237
98,196.0,17.09,0.735553,0.526728,5.884530e-10,1.390285,12.292444,0.441237
99,198.0,17.09,0.735553,0.526728,-3.513859e-09,1.390285,12.292444,0.441237


In [45]:
out.loc[0,'phi']

1.0

In [58]:
vessel_values_for_each_step.loc[6,'Resistance for Q']

6.0167730461175575

In [60]:
len(t)

101

In [65]:
vessel_data_alltime[0].loc[6,'Resistance for Q'] = 0

In [66]:
vessel_data_alltime[0]

,Name,Number,Diameter(µm),Length(µm),Pressure Drop(mmHg),Saturation,dS,Resistance for Q,Vt,partial pressure blood(mmHg),X-Area wall(mm^2?),wall thickness(µm),tissue partials(mmHg),Resistance for Q
0,A1,1,23.97,1267.6,6.93,0.9400,0.01000,0.248752,78005.271056,70.871373,438.213672,4.841407,68.497559,NaN
1,A2,2,19.17,930.3,5.87,0.9300,0.01000,0.421004,39040.035790,67.005340,313.016429,4.253658,65.226941,NaN
2,A3,4,15.28,543.6,4.02,0.9200,0.02000,0.576945,39019.312012,63.786724,228.546291,3.810683,60.367867,NaN
3,A4,8,12.08,302.3,2.70,0.8900,0.04000,0.775060,39016.335895,56.550348,170.463521,3.485850,49.437369,NaN
4,A5,16,9.46,161.2,1.82,0.8400,0.06250,1.049713,30341.627914,48.706014,130.572935,3.265971,36.295309,NaN
5,A6,32,7.32,154.7,2.35,0.7650,0.08750,2.700295,21321.745183,41.211431,103.110791,3.138293,29.926051,NaN
6,C,64,8.00,243.9,2.62,0.6650,0.07750,0.000000,9449.251212,34.556920,111.338044,3.172168,31.622967,8.2218
7,V6,32,11.51,473.9,1.27,0.6100,0.03375,1.463178,8202.351518,31.696836,161.197683,3.433626,30.710714,NaN
8,V5,16,14.53,272.3,0.61,0.5975,0.01125,0.348476,5514.003939,31.097186,214.009514,3.730525,30.104242,NaN
9,V4,8,17.79,426.6,0.89,0.5875,0.00750,0.256826,7277.302540,30.628584,281.309064,4.092089,29.879173,NaN


In [68]:
alpha*(baselines.loc[6,'Resistance for Q']/2)/out.loc[50,'phi']**4

8.215802746198753